In [ ]:
import os
import pygame

# 1. Instalar o driver da Adafruit
No terminal do Raspberry Pi:
```bash
sudo apt-get update
sudo apt-get upgrade
```
```bash
cd ~
curl -SLs https://apt.adafruit.com/add | sudo bash
sudo apt-get install -y adafruit-pitft-helper
sudo adafruit-pitft-helper
```

- 3.5" resistive display
- framebuffer (console ou X)
- touch

```bash
sudo reboot
```
```bash
ls /dev/fb*
```
Se aparecer /dev/fb1, este é o display TFT. O /dev/fb0 geralmente é o HDMI/desktop.


## 1.1. Verificar os dispositivos framebuffer conectados
No terminal da Raspberry Pi, execute:

```bash
ls /dev/fb*
``` 

Isso vai listar os dispositivos framebuffer disponíveis. Normalmente:
- /dev/fb0 é o framebuffer padrão, geralmente da saída HDMI ou display principal.

- /dev/fb1 (ou superior) geralmente é do display adicional (como um display touchscreen conectado via SPI ou outro barramento).

## 1.2. Habilitação do touch

Na instalação com adafruit-pitft-helper, você deve ter escolhido habilitar touch.
O touch será mapeado para algo como /dev/input/touchscreen ou /dev/input/eventX.

Calibração do toque:
Algumas vezes o toque pode estar descalibrado (ex: tocar no canto e o cursor ir para outro ponto). Se isso acontecer, você pode usar ts_calibrate:
```bash
sudo apt-get install tslib
sudo TSLIB_FBDEVICE=/dev/fb1 TSLIB_TSDEVICE=/dev/input/eventX ts_calibrate
```
Evite mostrar o cursor (não há ponteiro visível no touch):
```bash
pygame.mouse.set_visible(False)
```

# 2. Conexão Pygame - Dispaly


In [ ]:
os.environ["SDL_VIDEODRIVER"] = "fbcon"
os.environ["SDL_FBDEV"] = "/dev/fb1"  # ou fb0 se o display for principal
os.environ["SDL_MOUSEDRV"] = "TSLIB"
os.environ["SDL_MOUSEDEV"] = "/dev/input/touchscreen"

# 3. Código para os olhos da raposa

In [ ]:
pygame.init()

# Definição da tela
screen = pygame.display.set_mode((480, 320))
pygame.display.set_caption("Olhinhos de Raposa")

# Imagens da raposa
image_1r = pygame.image.load("olhos/1r.jpeg")
image_2r = pygame.image.load("olhos/2r.jpeg")
image_4r = pygame.image.load("olhos/4r.jpeg")
image_5r = pygame.image.load("olhos/5r.jpeg")

# Posição e estados iniciais
agent_x = 0
agent_y = 0
start_time = pygame.time.get_ticks()
image_changed_time = None
current_image = "5r"

# Mostrar a imagem inicial (5r)
screen.fill((200, 200, 200))
screen.blit(image_5r, (agent_x, agent_y))
pygame.display.flip()

# Imagens na Tela sob Loop
running = True
while running:
    print("Imagem atual:", current_image)  # 👈 aqui

    current_time = pygame.time.get_ticks()
    mouse_x, _ = pygame.mouse.get_pos()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # --- Lógica com base na posição do mouse ---
    if mouse_x < 100:
        if current_image != "1r":
            screen.fill((200, 200, 200))
            screen.blit(image_1r, (agent_x, agent_y))
            pygame.display.flip()
            current_image = "1r"
            image_changed_time = None 
            start_time = pygame.time.get_ticks() 
            # print("Imagem atual:", current_image) 


    elif mouse_x > 300:
        if current_image != "2r":
            screen.fill((200, 200, 200))
            screen.blit(image_2r, (agent_x, agent_y))
            pygame.display.flip()
            current_image = "2r"
            image_changed_time = None
            start_time = pygame.time.get_ticks()
            # print("Imagem atual:", current_image) 

    else:
        if current_image not in ["4r", "5r"]:
            screen.fill((200, 200, 200))
            screen.blit(image_5r, (agent_x, agent_y))
            pygame.display.flip()
            current_image = "5r"
            print("Imagem atual:", current_image)
            start_time = pygame.time.get_ticks()
            image_changed_time = None

        elif image_changed_time is None and current_time - start_time >= 5000:
            screen.fill((200, 200, 200))
            screen.blit(image_4r, (agent_x, agent_y))
            pygame.display.flip()
            current_image = "4r"
            print("Imagem atual:", current_image)
            image_changed_time = current_time

        elif current_image == "4r" and current_time - image_changed_time >= 1000:
            screen.fill((200, 200, 200))
            screen.blit(image_5r, (agent_x, agent_y))
            pygame.display.flip()
            current_image = "5r"
            print("Imagem atual:", current_image)
            start_time = pygame.time.get_ticks()
            image_changed_time = None

        # print("Imagem atual:", current_image)

pygame.quit()


# 4. Terminal
Certifique-se de que o script está executando fora do ambiente gráfico (X) se você escolheu o modo "Console" na configuração.

```bash
python3 seu_script.py
```